In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import pycountry_convert as pc
from matplotlib import rcParams
import geopandas as gpd

### Creación Dataset

In [2]:
life = pd.read_csv("life_expectancy_years.csv")
life = life.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Esperanza_vida")
income = pd.read_csv("income_per_person_gdppercapita_ppp_inflation_adjusted.csv")
income = income.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="PIB_capita")
poblacion = pd.read_csv("Population_total.csv")
poblacion = poblacion.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Población")
urban_pop = pd.read_csv("urban_population_percent_of_total.csv")
urban_pop = urban_pop.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Población_urbana")
children_woman = pd.read_csv("children_per_woman_total_fertility.csv")
children_woman = children_woman.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Bebés_por_mujer")
basic_san = pd.read_csv("at_least_basic_sanitation_overall_access_percent.csv")
basic_san = basic_san.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Saneamiento_básico")

In [3]:
df = income.merge(life,on=['country',"Date"],how="left")
df = df.merge(poblacion,on=['country',"Date"],how="left")
df = df.merge(urban_pop,on=['country',"Date"],how="left")
df = df.merge(children_woman,on=['country',"Date"],how="left")
df = df.merge(basic_san,on=['country',"Date"],how="left")
df = df.rename(columns={'country':'País','Date': 'Año', 'oldName2': 'newName2'})
df["Año"] = pd.to_numeric(df["Año"])

In [4]:
def alpha2code(country): #Obtiene codigo alpha2 del pais
    if country == "Cote d'Ivoire": country = "Ivory Coast"
    if country == "Congo, Dem. Rep.": return "CD"
    if country == "Congo, Rep.": return "CG"
    if country == "Micronesia, Fed. Sts.": country = "Micronesia"
    if country == "Hong Kong, China": country = "Hong Kong"
    if country == "Lao": country = "Laos"
    if country == "Timor-Leste": return "TL" #"TLS"
    if country == "St. Vincent and the Grenadines": return "VC" #"VCT"
    country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    return country_code

def getContinent(country): #Obtiene continente del pais
    if country == "Cote d'Ivoire": country = "Ivory Coast"
    if country == "Congo, Dem. Rep." or country == "Congo, Rep.": country = "Republic of the Congo"
    if country == "Micronesia, Fed. Sts.": country = "Micronesia"
    if country == "Hong Kong, China": country = "Hong Kong"
    if country == "Lao": country = "Laos"
    if country == "Timor-Leste": return "Asia"
    if country == "St. Vincent and the Grenadines": return "América Norte"
    country_code = pc.country_name_to_country_alpha2(country, cn_name_format="default")
    Continente_name = pc.country_alpha2_to_continent_code(country_code)
    nombreContinente = {"AS":"Asia","EU":"Europa","OC":"Oceanía","NA":"América Norte","SA":"América Sur","AF":"África"}
    Continente_name = nombreContinente[Continente_name]
    if(Continente_name=="NA"): print(country)
    return Continente_name


def value_to_float(x): # Los números vienen con sufijos para cuantificar
    if type(x) == float:
        return x
    if type(x) == int:
        return float(x)
    if 'k' in x:
            return float(x.replace('k', '')) * 1000
    if 'M' in x:
            return float(x.replace('M', '')) * 1000000
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return float(x)

In [5]:
dfG = gpd.read_file("World_Countries_(Generalized).geojson")
dfG.set_geometry("geometry")
# Anexamos algunos territorios controlados por países
dfG = dfG.replace("ESH", "MAR")
dfG = dfG.replace("XXZ", "IND")
dfG = dfG.replace("XXY", "IND")

In [6]:
f = lambda x: getContinent(x)
f2 = lambda x: value_to_float(x)
f3 = lambda x: alpha2code(x)

#Procesamos datos
df['Continente'] = df['País'].apply(f)
df['PIB_capita'] = df['PIB_capita'].apply(f2)
df['Población'] = df['Población'].apply(f2)
df['ISO'] = df['País'].apply(f3)

### Procesado Trabajo

In [7]:
df = df[df["Año"] < 2021] #Años posteriores tienen muchas lagunas en los datos
#Sustituimos nombres a español (chatGPT)
countryDict= {'Afghanistan': 'Afganistán', 'Albania': 'Albania', 'Algeria': 'Argelia', 'Andorra': 'Andorra', 'Angola': 'Angola', 'Antigua and Barbuda': 'Antigua y Barbuda', 'Argentina': 'Argentina', 'Armenia': 'Armenia', 'Australia': 'Australia', 'Austria': 'Austria', 'Azerbaijan': 'Azerbaiyán', 'Portugal': 'Portugal', 'Bahamas': 'Bahamas', 'Bahrain': 'Baréin', 'Bangladesh': 'Bangladesh', 'Barbados': 'Barbados', 'Belarus': 'Bielorrusia', 'Belgium': 'Bélgica', 'Belize': 'Belice', 'Benin': 'Benín', 'Bhutan': 'Bután', 'Bolivia': 'Bolivia', 'Bosnia and Herzegovina': 'Bosnia y Herzegovina', 'Botswana': 'Botsuana', 'Brazil': 'Brasil', 'Brunei': 'Brunéi', 'Bulgaria': 'Bulgaria', 'Burkina Faso': 'Burkina Faso', 'Burundi': 'Burundi', 'Cape Verde': 'Cabo Verde', 'Cambodia': 'Camboya', 'Cameroon': 'Camerún', 'Canada': 'Canadá', 'Spain': 'España', 'Central African Republic': 'República Centroafricana', 'Chad': 'Chad', 'Chile': 'Chile', 'China': 'China', 'Colombia': 'Colombia', 'Comoros': 'Comoras', 'Congo, Rep.': 'República del Congo', 'Congo, Dem. Rep.': 'República Democrática del Congo', 'Costa Rica': 'Costa Rica', "Cote d'Ivoire": "Costa de Marfil", 'Croatia': 'Croacia', 'Cuba': 'Cuba', 'Cyprus': 'Chipre', 'Czech Republic': 'República Checa', 'Denmark': 'Dinamarca', 'Djibouti': 'Yibuti', 'Dominica': 'Dominica', 'Dominican Republic': 'República Dominicana', 'Ecuador': 'Ecuador', 'Egypt': 'Egipto', 'El Salvador': 'El Salvador', 'Equatorial Guinea': 'Guinea Ecuatorial', 'Eritrea': 'Eritrea', 'Estonia': 'Estonia', 'Eswatini': 'Esuatini', 'Ethiopia': 'Etiopía', 'Fiji': 'Fiyi', 'Finland': 'Finlandia', 'France': 'Francia', 'Gabon': 'Gabón', 'Gambia': 'Gambia', 'Georgia': 'Georgia', 'Germany': 'Alemania', 'Ghana': 'Ghana', 'Greece': 'Grecia', 'Grenada': 'Granada', 'Guatemala': 'Guatemala', 'Guinea': 'Guinea', 'Guinea-Bissau': 'Guinea-Bisáu', 'Guyana': 'Guyana', 'Haiti': 'Haití', 'Honduras': 'Honduras', 'Hungary': 'Hungría', 'Iceland': 'Islandia', 'India': 'India', 'Indonesia': 'Indonesia', 'Iran': 'Irán', 'Iraq': 'Irak', 'Ireland': 'Irlanda', 'Israel': 'Israel', 'Italy': 'Italia', 'Jamaica': 'Jamaica', 'Japan': 'Japón', 'Jordan': 'Jordania', 'Kazakhstan': 'Kazajistán', 'Kenya': 'Kenia', 'Kiribati': 'Kiribati', 'Kuwait': 'Kuwait', 'Kyrgyz Republic': 'Kirguistán', 'Lao': 'Laos', 'Latvia': 'Letonia', 'Lebanon': 'Líbano', 'Lesotho': 'Lesoto', 'Liberia': 'Liberia', 'Libya': 'Libia', 'Lithuania': 'Lituania', 'Luxembourg': 'Luxemburgo', 'Madagascar': 'Madagascar', 'Malawi': 'Malaui', 'Malaysia': 'Malasia', 'Maldives': 'Maldivas', 'Mali': 'Malí', 'Malta': 'Malta', 'Marshall Islands': 'Islas Marshall', 'Mauritania': 'Mauritania', 'Mauritius': 'Mauricio', 'Mexico': 'México', 'Micronesia, Fed. Sts.': 'Micronesia', 'Moldova': 'Moldavia', 'Monaco': 'Mónaco', 'Mongolia': 'Mongolia', 'Montenegro': 'Montenegro', 'Morocco': 'Marruecos', 'Mozambique': 'Mozambique', 'Myanmar': 'Myanmar', 'Namibia': 'Namibia', 'Nauru': 'Nauru', 'Nepal': 'Nepal', 'Netherlands': 'Países Bajos', 'New Zealand': 'Nueva Zelanda', 'Nicaragua': 'Nicaragua', 'Niger': 'Níger', 'Nigeria': 'Nigeria', 'North Korea': 'Corea del Norte', 'North Macedonia': 'Macedonia del Norte', 'Norway': 'Noruega', 'Oman': 'Omán', 'Pakistan': 'Pakistán', 'Palau': 'Palaos', 'Palestine': 'Palestina', 'Panama': 'Panamá', 'Papua New Guinea': 'Papúa Nueva Guinea', 'Paraguay': 'Paraguay', 'Peru': 'Perú', 'Philippines': 'Filipinas', 'Poland': 'Polonia', 'Qatar': 'Catar', 'Romania': 'Rumania', 'Russia': 'Rusia', 'Rwanda': 'Ruanda', 'St. Kitts and Nevis': 'San Cristóbal y Nieves', 'St. Lucia': 'Santa Lucía', 'St. Vincent and the Grenadines': 'San Vicente y las Granadinas', 'Samoa': 'Samoa', 'San Marino': 'San Marino', 'Sao Tome and Principe': 'Santo Tomé y Príncipe', 'Saudi Arabia': 'Arabia Saudita', 'Senegal': 'Senegal', 'Serbia': 'Serbia', 'Seychelles': 'Seychelles', 'Sierra Leone': 'Sierra Leona', 'Singapore': 'Singapur', 'Slovak Republic': 'República Eslovaca', 'Slovenia': 'Eslovenia', 'Solomon Islands': 'Islas Salomón', 'Somalia': 'Somalia', 'South Africa': 'Sudáfrica', 'South Korea': 'Corea del Sur', 'South Sudan': 'Sudán del Sur', 'Sri Lanka': 'Sri Lanka', 'Sudan': 'Sudán', 'Suriname': 'Surinam', 'Sweden': 'Suecia', 'Switzerland': 'Suiza', 'Syria': 'Siria', 'Tajikistan': 'Tayikistán', 'Tanzania': 'Tanzania', 'Thailand': 'Tailandia', 'Timor-Leste': 'Timor Oriental', 'Togo': 'Togo', 'Tonga': 'Tonga', 'Trinidad and Tobago': 'Trinidad y Tobago', 'Tunisia': 'Túnez', 'Turkey': 'Turquía', 'Turkmenistan': 'Turkmenistán', 'Tuvalu': 'Tuvalu', 'Uganda': 'Uganda', 'Ukraine': 'Ucrania', 'United Arab Emirates': 'Emiratos Árabes Unidos', 'United Kingdom': 'Reino Unido', 'United States': 'Estados Unidos', 'Uruguay': 'Uruguay', 'Uzbekistan': 'Uzbekistán', 'Vanuatu': 'Vanuatu', 'Venezuela': 'Venezuela', 'Vietnam': 'Vietnam', 'Yemen': 'Yemen', 'Zambia': 'Zambia', 'Zimbabwe': 'Zimbabue'}
df = df.replace({"País": countryDict})
df.to_csv("metricas_globales.csv",encoding="utf-8") #Guardamos

In [8]:
#Sustituimos nombres a español geoDataFrame (chatGPT)
dfG["COUNTRY"] = ['Afganistán', 'Albania', 'Argelia', 'Samoa Americana', 'Andorra', 'Angola', 'Anguila', 'Antártida', 'Antigua y Barbuda', 'Argentina', 'Armenia',
                'Aruba', 'Australia', 'Austria', 'Azerbaiyán', 'Azores', 'Bahamas', 'Baréin', 'Bangladesh', 'Barbados', 'Bielorrusia', 'Bélgica', 'Belice', 'Benín',
                'Bermudas', 'Bután', 'Bolivia', 'Bonaire', 'Bosnia y Herzegovina', 'Botsuana', 'Isla Bouvet', 'Brasil', 'Territorio Británico del Océano Índico', 'Islas Vírgenes Británicas',
                'Brunéi', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Camboya', 'Camerún', 'Canadá', 'Canarias', 'Islas Caimán', 'República Centroafricana', 'Chad', 'Chile',
                'China', 'Isla de Navidad', 'Islas Cocos', 'Colombia', 'Comoras', 'Congo', 'República Democrática del Congo', 'Islas Cook', 'Costa Rica', "Costa de Marfil", 'Croacia',
                'Cuba', 'Curazao', 'Chipre', 'República Checa', 'Dinamarca', 'Yibuti', 'Dominica', 'República Dominicana', 'Ecuador', 'Egipto', 'El Salvador', 'Guinea Ecuatorial', 
                'Eritrea', 'Estonia', 'Esuatini', 'Etiopía', 'Islas Malvinas', 'Islas Feroe', 'Fiyi', 'Finlandia', 'Francia', 'Guayana Francesa', 'Polinesia Francesa',
                'Territorios Australes Franceses', 'Gabón', 'Gambia', 'Georgia', 'Alemania', 'Ghana', 'Gibraltar', 'Islas Gloriosas', 'Grecia', 'Groenlandia', 'Granada',
                'Guadalupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haití', 'Isla Heard e Islas McDonald', 'Honduras', 'Hungría', 'Islandia',
                'India', 'Indonesia', 'Irán', 'Irak', 'Irlanda', 'Isla de Man', 'Israel', 'Italia', 'Jamaica', 'Japón', 'Jersey', 'Jordania', 'Isla Juan de Nova', 'Kazajistán',
                'Kenia', 'Kiribati', 'Kuwait', 'Kirguistán', 'Laos', 'Letonia', 'Líbano', 'Lesoto', 'Liberia', 'Libia', 'Liechtenstein', 'Lituania', 'Luxemburgo', 'Madagascar',
                'Madeira', 'Malaui', 'Malasia', 'Maldivas', 'Malí', 'Malta', 'Islas Marshall', 'Martinica', 'Mauritania', 'Mauricio', 'Mayotte', 'México', 'Micronesia', 'Moldavia',
                'Mónaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Marruecos', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Países Bajos', 'Nueva Caledonia',
                'Nueva Zelanda', 'Nicaragua', 'Níger', 'Nigeria', 'Niue', 'Isla Norfolk', 'Corea del Norte', 'Macedonia del Norte', 'Islas Marianas del Norte', 'Noruega',
                'Omán', 'Pakistán', 'Palau', 'Territorio Palestino', 'Panamá', 'Papúa Nueva Guinea', 'Paraguay', 'Perú', 'Filipinas', 'Islas Pitcairn', 'Polonia',
                'Portugal', 'Puerto Rico', 'Catar', 'Reunión', 'Rumania', 'Federación de Rusia', 'Ruanda', 'Saba', 'San Bartolomé', 'San Eustaquio', 'Santa Elena',
                'San Cristóbal y Nieves', 'Santa Lucía', 'San Martín', 'San Pedro y Miquelón', 'San Vicente y las Granadinas', 'Samoa', 'San Marino',
                'Santo Tomé y Príncipe', 'Arabia Saudita', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leona', 'Singapur', 'Sint Maarten', 'Eslovaquia',
                'Eslovenia', 'Islas Salomón', 'Somalia', 'Sudáfrica', 'Islas Georgias del Sur y Sandwich del Sur', 'Corea del Sur', 'Sudán del Sur',
                'España', 'Sri Lanka', 'Sudán', 'Surinam', 'Svalbard', 'Suecia', 'Suiza', 'Siria', 'Tayikistán', 'Tanzania', 'Tailandia', 'Timor Oriental',
                'Togo', 'Tokelau', 'Tonga', 'Trinidad y Tobago', 'Túnez', 'Turquía', 'Turkmenistán', 'Islas Turcas y Caicos', 'Tuvalu', 'Uganda', 'Ucrania',
                'Emiratos Árabes Unidos', 'Reino Unido', 'Estados Unidos', 'Islas menores alejadas de los Estados Unidos', 'Uruguay',
                'Islas Vírgenes de los Estados Unidos', 'Uzbekistán', 'Vanuatu', 'Ciudad del Vaticano', 'Venezuela', 'Vietnam', 'Wallis y Futuna',
                'Yemen', 'Zambia', 'Zimbabue']
dfG["geometry"] = dfG.simplify(0.1) #0.1 es el mejor valor encontrado entre calidad/fluidez
dfG.to_file("geografia.geojson")